In [14]:
#load data
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "GNA_bk_1850.txt"
with open(file_path, 'r', encoding="cp1252") as f: # The encoding was different from the previous 2
  data = f.readlines()

In [ ]:
import pandas as pd
pd.DataFrame(data).shape

(17089, 1)

In [15]:
# Remove first 8 lines of the Entry
data = data[8:17089]

Data cleaning

In [16]:
# Remove Headers from data
header = ['BROOKLYN DIRECTORY.\n', 'BROOKLYN DIRECTORY,\n', 'BROOKLYN DIR-ECTORY,\n', 'BROOKLYN DIRECTOE.Y.\n', 'BROOKLYN DIBECTORY.\n', 'BROOKLYN DIR-ECTORY\n', 'BROOKLYN BIRECTORY,\n', 'BROOK.LYN DIRECTORY.', '160 BROOKLYN DIRECTORY.', 'BROOKLYN HIRECTORY.', 'BROOKLYN DIRECTOHY.', 'BROOKLYN tlKECTORY.', 'HIIOOKLYN DIRECTORY.', 'BROOKLYN DIKECTOKY.', 'BH00KL1N DIRECTORY.',
          'BROOKLYN DIRECTORT.',
'BEOOKLYN DIRECTORY.',
'BROOKLYN DIRECTORY',
'BROOKLYN DIHECTOR Y.',
'BROOKLYN D1EECTORY.',
"BROOKLYN DI E ECTOK T,",
'BROOKLYN MRECTOEY.',
'BROOKLYN PIKECTORV.',
'B KOOK LYN DIRECTORY,',
'BROOKLYN DISECTOBY.',
"BRO'OKLYN DIRECTORY.",
'BROOKLYN DIRECTOEY.',
"'BR,OOKLYN DIRECTOR. Y.",
'BROOKLYN DIKECTORY.',
"BROOKLYN D l R E C T 0 K Y,",
'BROOKLYN PLRECTORY.',
'BROOKLYN DIEECTORY.',
'BROOKLYN DIKECTORI’.',
'BROOKLYN HIRECTORY.',
'BROOKLYN B IE ECTOK Y.',
'BROOKLYN DIBECTOE.Y.',
'BROOKLYN DIBECTOKY.',
'BBOOKLYN DIRECTOKY.',
'BROOKLYN DIBECTOKY.',
'BROOKLYN DIKECTORY.',

]
data = [entry for entry in data if entry not in header]

# Remove \n
data = [entry.replace('\n', '') for entry in data]

# Remove \t
data = [entry.replace('\t', ' ') for entry in data]

# Remove Blanks
data = [entry for entry in data if entry != ' ']

# Remove Page Numbers
import re
data = [entry for entry in data if not re.search('^[0-9]+$', entry)]

In [17]:
# Remove Headers from data
header = ['BEOOKLYN DIRECTORY.',
'BROOKLYN DIRECTORY',
'BROOKLYN DIHECTOR Y.',
'BROOKLYN D1EECTORY.',
"BROOKLYN DI E ECTOK T,",
'BROOKLYN MRECTOEY.',
'BROOKLYN PIKECTORV.',
'B KOOK LYN DIRECTORY,',
'BROOKLYN DISECTOBY.',
"BRO'OKLYN DIRECTORY.",
'BROOKLYN DIRECTOEY.',
"'BR,OOKLYN DIRECTOR. Y.",
'BROOKLYN DIKECTORY.',
"BROOKLYN D l R E C T 0 K Y,",
'BROOKLYN PLRECTORY.',
'BROOKLYN DIEECTORY.',
'BROOKLYN DIKECTORI’.',
'BROOKLYN HIRECTORY.',
'BROOKLYN B IE ECTOK Y.',
'BROOKLYN DIBECTOE.Y.',
'BROOKLYN DIBECTOKY.',
'BBOOKLYN DIRECTOKY.',
'BROOKLYN DIKECTORY',
'BROOKLYN DIRECTORS',
"BROOKL'YN DIRECTORY,",
'BROOKLYN DIRECTORT.',
'BROOKLYN DIKECTORY,',
'BROOKLYN BIRECTORY.',
'BROOKLYN E I B E C T 0 It tr .'
'BROOKLYN DIRECTORF.'

]
data = [entry for entry in data if entry not in header]


In [18]:
# Remove any BROOKLYN DIRECTORY. Page number entries
import re
for index in range(0, len(data)):
    clear = re.findall(r'^BROOKLYN DIRECTORY. [0-9]+$', data[index])
    if clear:
        data[index] = ""
    else:
        pass

In [19]:
# Change 'IFF' to N
for index in range(0, len(data)):
    n = re.findall('^Iff$', data[index])
    if n:
        data[index] = "N"
    else:
        pass

In [ ]:
len(data)

16071

Moving the [... down to the next row

In [20]:
import re
for index in range(0, len(data)):
    match = re.findall(r'\[.*$', data[index])
    if match:
        data[index + 1] = data[index + 1] + ' ' + str(match[0]).strip('[')
        data[index] = data[index].replace(str(match[0]), '')
    else:
        pass

len(data)

16035

Issues with Paranthesis

*   Adams J W, civil engineer Hoyt n State (Classon av
*   Addy Edward 50 Middagh (tionary 262 Fulton


Resolve By:
*   If the length of the line before is longer than the length of the line after, add the (... to the line before
  * vice versa


In [21]:

for index in range(0, len(data)):
    match = re.findall(r'\(.*$', data[index])
    if match:
        if len(data[index + 1]) > len(data[index - 1]):
            data[index + 1] = data[index + 1] + ' ' + str(match[0]).strip('(')
            data[index] = data[index].replace(str(match[0]), '')
        else:
            data[index - 1] = data[index - 1] + ' ' + str(match[0]).strip('(')
            data[index] = data[index].replace(str(match[0]), '')
    else:
        pass

In [22]:
# Remove those Sta- ionary types
for index in range(0, len(data)):
    data[index] = data[index].replace('- ', '')


Occupations need to be separated from Address by ,

In [23]:

# Deal with one word Occupations
pattern1 = ', [a-z]+ [0-9A-Z]' # If occupation is 1 word
for index in range(0, len(data)):
    if re.findall(pattern1, data[index]):
        x = re.findall(pattern1, data[index])[0]
        y = x.replace(' ', ', ')
        z = y.replace(',,', ',')
        data[index] = data[index].replace(x, z)
    else:
        pass

In [24]:
# Deal with two word Occupations
pattern2 = ', [a-z]+ [a-z]* [0-9A-Z]' # If occupation is 2 words
for index in range(0, len(data)):
    if re.findall("widow", data[index]): # Ignore widow cases
        pass
    else:
        if re.findall(pattern2, data[index]):
            x = re.findall(pattern2, data[index])[0]
            y = re.findall(' [0-9A-Z]', x)[0]
            z = re.sub(y, ',' + y, x)
            data[index] = data[index].replace(x, z)
        else:
            pass



In [25]:
# Deal with 3 word Occupations
pattern3 = ', [a-z]+ [a-z]* [a-z]* [0-9A-Z]' # If occupation is 3 words
for index in range(0, len(data)):
    if re.findall("widow", data[index]): # Ignore widow cases
        pass
    else:
        if re.findall(pattern3, data[index]):
            x = re.findall(pattern3, data[index])[0]
            y = re.findall(' [0-9A-Z]', x)[0]
            z = re.sub(y, ',' + y, x)
            data[index] = data[index].replace(x, z)
        else:
            pass

In [26]:
# Deal with 4 word Occupations
pattern4 = ', [a-z]+ [a-z]* [a-z]* [a-z]* [0-9A-Z]' # If occupation is 4 words
for index in range(0, len(data)):
    if re.findall("widow", data[index]): # Ignore widow cases
        pass
    else:
        if re.findall(pattern4, data[index]):
            x = re.findall(pattern4, data[index])[0]
            y = re.findall(' [0-9A-Z]', x)[0]
            z = re.sub(y, ',' + y, x)
            data[index] = data[index].replace(x, z)
        else:
            pass


In [27]:
# Deal with 5 word Occupations
pattern5 = ', [a-z]+ [a-z]* [a-z]* [a-z]* [a-z]* [0-9A-Z]' # If occupation is 5 words
for index in range(0, len(data)):
    if re.findall("widow", data[index]): # Ignore widow cases
        pass
    else:
        if re.findall(pattern5, data[index]):
            x = re.findall(pattern5, data[index])[0]
            y = re.findall(' [0-9A-Z]', x)[0]
            z = re.sub(y, ',' + y, x)
            data[index] = data[index].replace(x, z)
        else:
            pass

In [28]:
# Deal with 6 word Occupations
pattern6 = ', [a-z]+ [a-z]* [a-z]* [a-z]* [a-z]* [a-z]* [0-9A-Z]' # If occupation is 6 words
for index in range(0, len(data)):
    if re.findall("widow", data[index]): # Ignore widow cases
        pass
    else:
        if re.findall(pattern6, data[index]):
            x = re.findall(pattern6, data[index])[0]
            y = re.findall(' [0-9A-Z]', x)[0]
            z = re.sub(y, ',' + y, x)
            data[index] = data[index].replace(x, z)
        else:
            pass



In [29]:
# Deal with 7 word Occupations
pattern7 = ', [a-z]+ [a-z]* [a-z]* [a-z]* [a-z]* [a-z]* [a-z]* [0-9A-Z]' # If occupation is 7 words
for index in range(0, len(data)):
    if re.findall("widow", data[index]): # Ignore widow cases
        pass
    else:
        if re.findall(pattern7, data[index]):
            x = re.findall(pattern7, data[index])[0]
            y = re.findall(' [0-9A-Z]', x)[0]
            z = re.sub(y, ',' + y, x)
            data[index] = data[index].replace(x, z)
        else:
            pass



In [54]:
# remove special characters from the start of the sentence
for index in range(0, len(data)):
    if (len(data[index]) == 0):
      pass
    elif (data[index][0] in ("'",",","^","!","_","-"," ","’")):
      data[index] = data[index][1:]
    elif (data[index][0] in ("—")):
      data[index] = data[index][1:]
    else:
      pass
    

In [31]:
for index in range(0, len(data)):
    print(data[index])

Streaming output truncated to the last 5000 lines.
O'Brien William, machinist, Pearl c Front
O'Brien William, blacksmith, 212 Jay
O
0‘Callaghan Peter, laborer, Tillary n Gold Brien William, moulder Front c Pearl
Ochs Christopher, shoemaker, 19 Dean
Ochs Frederick, shoemaker, 19 Dean
O'Connell Joseph, shoemaker, Plymouth n Bridge
O'Connell Thomas, sawyer, Water n Gold
O‘Connell YVilliam, rigger, Water c Dock
O*Connor Ann, widow of James, Columbia c Pacific
O'Connor Dennis, laborer, Columbia c Atlantic
O'Connor Dennis, laborer, Willoughby c Hudson av
O'Connor Dennis, hat and cap store, 57 Main
O'Connor F'elix, laborer, Bergen c Smith
O'Connor James, mer. Old slip N Y h 56 State
0‘Connor John, currier r, 216 Gold
O“Connor Michael R, com. agent 19 Atlantic t
O'Connor Michael, Columbia c Summi
O'Connor Patrick, sawyer, Water n Jay
O'Connor Terence, shoemaker, 86 Smith
O'Connell B, coachman, 205 Myrtle av
Oddie Walter M, artist, Bedford
O‘Deay Dennis, bootmaker, Columbia n Atlantic
O‘Deay Pa


Divide records into different sections based on first letter

In [55]:
d = {}
value = []
i = 0
for line in data:
  line = line.strip()
  if line:
    if len(line) == 1: 
      if re.match("^[A-Z]$", line):
        # save previous value
        if value:  
          if value[0] == "'":
            value = value[1:]

          d.setdefault(key, [])
          if value[2][0] != key :
            if value[2][0] == '0':
              key = 'O'
            else:
              key = value[2][0]
          
          d[key].append(value)
        
        key = line
        value = []
    else:
      value.append(line)


d.setdefault(key, [])
d[key].append(value)

In [56]:
for key, value in d.items():
  d[key] = [item for sublist in d[key] for item in sublist]

Combline lines that belong to the same record
Criteria:

1) First letter of next line not equal to the initial letter in this section

2) Special cases in the start of next line: B’klyn, B'way, E., W.

3) Do not start with 0' or 0‘

In [57]:
for key,value in d.items():
  initial = key[0]
  new_value = []
  n = len(value)
  i = 0
  
  while i < n:
    if i < n-1:
      if (value[i+1][0] != initial and value[i+1][0] != "*" and value[i+1][0] != "." and (value[i+1][0:2] not in ( "0'", "0‘"))) or (value[i+1][0:2] in ["B’", "E.", "W."]) or (value[i+1][0:2] == ". "):
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2
      else:
        new_value.append(value[i])
        i += 1 
    else:
      if value[i][0] == initial:
        new_value.append(value[i])
      i += 1
  
  d[key] = new_value

In [46]:
#remove \"(see adv....\"\n",
for key,value in d.items():
    n = len(value)
    for i in range(n):
      ls = value[i].split()
      n2 = len(ls)
      for j in range(n2):
        if (("(see" in ls[j]) or ("(See" in ls[j])):
          value[i] = ' '.join(ls[:j])
          break

Identify business vs person records

Made an update on 27/03/20 to change the index notation.

The first letter of Company names were being removed by the function
ZUCCA BROTHERS
ARNAULT PETER
APPELL JACOB
APPLETON’S AMERICAN CYCLOPG5DIA
BURTON A. B

In [58]:
# add an index to each record
# in another dictionary, for each index specify whether it's business or person record
index = 1
d_bp = []
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    if ls:
      if (ls[0][0] == "*" ):
        value[i] = value[i][1:]
        d_bp.append([str(index), "b"])
      elif (len(ls[0]) > 1 and ls[0].isupper()):
        d_bp.append([str(index), "b"])
      else:
        d_bp.append([str(index), "p"])
    
      value[i] = str(index) + "\t" + value[i]
      index += 1

Save files

In [59]:
file = open("nypl_1850_bk_clean_records.txt", "w")
for key,value in d.items():
  file.write(key + '\n')
  for record in value:
    file.write(record + '\n')
  file.write('\n')

file.close()

file2 = open("bk_index_business_person.txt", "w")
for item in d_bp:
  file2.write(item[0] + ' ' + item[1] + '\n')
  
file2.close()

import the old cleaned file to compare

In [60]:
# import the old cleaned file to compare
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "/content/nypl_1850_bk_starred_clean_old.txt"
with open(file_path) as f:
  data_clean = f.readlines()


import re
n = len(data_clean)
full_entry = []
for i in range(n):
  ls = data_clean[i].split()
  if ls:
    if bool(re.match("([A-Z]+\.)", ls[0])) != True:
      full_entry.append(' '.join(ls[1:]))

import the new cleaned file

In [61]:
import re
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "/content/nypl_1850_bk_clean_records.txt"
with open(file_path) as f:
  new_output = f.readlines()

n = len(new_output)
new_output_check = []
for i in range(n):
  ls = new_output[i].split()
  if ls:
    if bool(re.match("([A-Z]+\.)", ls[0])) != True:
      new_output_check.append(' '.join(ls[1:]))

check how many entries do not match with the old cleaned file

In [62]:
left_new_entries = []

for i in new_output_check:
  if i in full_entry:
    continue
  else:
    left_new_entries.append(i)


print(len(new_output_check)) # records in new file
print(len(full_entry))      # records in old file
print(len(left_new_entries))  # extra records

15382
10756
5097


There are an extra of ~5000 records which were earlier getting mixed because of overwrite in the dictionary of records

In [63]:
left_new_entries

['Abrahams Thompson, seaman, Clark’s alley n York',
 'Abrams Thomas, laborer, 74 Jay',
 'Adams Robert, waiter, 43 Talman',
 'Anderson Barrett, porter, 64 Jay',
 'Anderson William, laborer, 64 Jay',
 'Ash Jane, widow, 41 Hicks c Middagh',
 'Avery R H, clerk, Adelphi n Jamaica av',
 'Avery Samuel P, wood engraver, 129 Fulton N Y h 114 Smith',
 'Bagley George, laborer, 285 Hudson av',
 'Bailey Benjamin, waiter, Hart’s alley n Gold',
 'Banks Peter, laborer, 266 Marshall',
 'Banyan Benjamin, laborer, Pacific n Court',
 'Bayard Margaret, widow, 18 Ilartt’s Alley',
 'Beekman Andrew, whitewashcr r, 56 Stanton',
 'Bisbee Eliza, boarding house, 3d av n 18th st Gowanus',
 'Bisbee L, Sec. U S Life Ins Co 65 Lawrence',
 'Black G C, watchmaker, 42 Fulton N Y h 134 Adams',
 'Black John, master’s mate U s N 54 Hudson av',
 'Blake William, steward, 35 chapel',
 'Boden F W, vinegar manufacturer, Boerum n Dean',
 'Bodge William, printer, 110 Willoughby',
 'BoerumPeter, waiter, 99 High',
 'Bowen Plato, wh

In [64]:
file = open("/content/1850_bk_diff_new_entry.txt", "w")
for record in left_new_entries:
  file.write(record + '\n')
  file.write('\n')

file.close()

check what kind of records present in old file which are not in the newly cleaned file

In [65]:
left_old_entries = []

for i in full_entry:
  if i in new_output_check:
    continue
  else:
    left_old_entries.append(i)


print(len(new_output_check)) # records in new file
print(len(full_entry))      # records in old file
print(len(left_old_entries))  # old records not present in new file

15382
10756
473


In [66]:
left_old_entries

['*Abrahams Thompson, seaman, Clark’s alley n York',
 '*Abrams Thomas, laborer, 74 Jay',
 '*Adams Robert, waiter, 43 Talman',
 '*Anderson Barrett, porter, 64 Jay',
 '*Anderson William, laborer, 64 Jay',
 '*Ash Jane, widow, 41 Hicks c Middagh',
 'Avery R H, clerk, Adelphi n Jamaica av -—Avery Samuel P, wood engraver, 129 Fulton N Y h 114 Smith',
 '*Bagley George, laborer, 285 Hudson av',
 '*Bailey Benjamin, waiter, Hart’s alley n Gold',
 '*Banks Peter, laborer, 266 Marshall',
 '*Banyan Benjamin, laborer, Pacific n Court',
 '*Bayard Margaret, widow, 18 Ilartt’s Alley',
 '*Beekman Andrew, whitewashcr r, 56 Stanton',
 "Bisbee Eliza, boarding house, 3d av n 18th st Gowanus 'Bisbee L, Sec. U S Life Ins Co 65 Lawrence",
 "Black G C, watchmaker, 42 Fulton N Y h 134 Adams 'Black John, master’s mate U s N 54 Hudson av",
 '*Blake William, steward, 35 chapel',
 'Boden F W, vinegar manufacturer, Boerum n Dean ,Bodge William, printer, 110 Willoughby',
 '*BoerumPeter, waiter, 99 High',
 'BROOKLYN DIR

There might be still be scope of improvement like identifying more mispelled Brooklyn Directory keywords